In [1]:
%%configure -f
{"executorMemory":"5G", "numExecutors":2, "executorCores":3, "conf": {"spark.dynamicAllocation.enabled": "false"}}

In [2]:
val bucketname = "unibo-bd2122-vmattiussi"

val path_ml_tripdata = "s3a://"+bucketname+"/datasets/project/2018/yellow_tripdata_2018-01.parquet" //"s3a://"+bucketname+"/datasets/project/2017/"
val path_ml_tripdata802 = "s3a://"+bucketname+"/datasets/project/2018/yellow_tripdata_2018-02.parquet" //"s3a://"+bucketname+"/datasets/project/2016/"

sc.applicationId

"SPARK UI: Enable forwarding of port 20888 and connect to http://localhost:20888/proxy/" + sc.applicationId + "/"

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1658344938261_0001,spark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

bucketname: String = unibo-bd2122-vmattiussi
path_ml_tripdata: String = s3a://unibo-bd2122-vmattiussi/datasets/project/2018/yellow_tripdata_2018-01.parquet
path_ml_tripdata802: String = s3a://unibo-bd2122-vmattiussi/datasets/project/2018/yellow_tripdata_2018-02.parquet
res3: String = application_1658344938261_0001
res5: String = SPARK UI: Enable forwarding of port 20888 and connect to http://localhost:20888/proxy/application_1658344938261_0001/


# Dataset
Il dataset contiene dati sui viaggi effetuati dai "yellow taxi" di new york negli anni 2018-2017-2016.
I dati sono presenti nel formato Parquet; dopo essere stati importati, tutti i job sono stati fatti dopo una successiva conversione in RDD.
I dati sono raggruppati mensilmente, per un totale di 36 file Parquet.
Link: https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page

Importazione dei dati in formato Parquet.

In [3]:
import spark.implicits._
val parquetFileDF = spark.read.parquet(path_ml_tripdata)
val parquetFileDF802 = spark.read.parquet(path_ml_tripdata802)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import spark.implicits._
parquetFileDF: org.apache.spark.sql.DataFrame = [VendorID: bigint, tpep_pickup_datetime: timestamp ... 17 more fields]
parquetFileDF802: org.apache.spark.sql.DataFrame = [VendorID: bigint, tpep_pickup_datetime: timestamp ... 17 more fields]


## Analisi dei campi del dataset
- pickup: timestamp del momento dell'inizio del viaggio.
- dropoff: timestamp del momento della fine del viaggio.
- passenger: numero di clienti passeggeri.
- distance: distanza totale in km percorsa dal taxi per questo viaggio.
- startloc: zona di partenza del viaggio.
- endloc: zona di arrivo del viaggio.
- fare: prezzo del viaggio.

In [4]:
import java.sql.Timestamp
case class TaxiTrip(
  pickup:Timestamp,
  dropoff:Timestamp,
  passenger:Long,
  distance:Double,
  startloc:Long,
  endloc:String,
  fare:Double
)

object TaxiTrip {
    def extract(row:org.apache.spark.sql.Row) = {
        val pickup = row.getTimestamp(1)
        val dropoff = row.getTimestamp(2)
        val passenger = row.getLong(3)
        val distance = row.getDouble(4)
        val startloc = row.getLong(5)
        val endloc = row.getString(6)
        val fare = row.getDouble(10)

        new TaxiTrip(pickup,dropoff,passenger,distance,startloc,endloc,fare)
    }
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import java.sql.Timestamp
defined class TaxiTrip
defined object TaxiTrip
Companions must be defined together; you may wish to use :paste mode for this.


La seguente funzione serve per convertire da formato timestamp a calendar, in modo da poter estrarre informazioni come giorno del mese o ora del giorno.

In [5]:
import java.util.Calendar
def getDayTime(time: Long): Calendar = {
    var date:Calendar = Calendar.getInstance();
    date.setTimeInMillis(time);
    date
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import java.util.Calendar
getDayTime: (time: Long)java.util.Calendar


### Creazione e caching del RDD a partire dal file Parquet.
Caching per migliorare le prestazioni in vista di futuri utilizzi.
Si è optato per la scelta tenere due ripartizioni (quindi due task per core).
Essendoci due executor con tre core ciascuno, il numero totale di core è sei.
Quindi si è scelta una ripartizione di 12, anche se guardando i tempi di esecuzione il miglioramento è solo di qualche secondo.

In [6]:
import org.apache.spark.HashPartitioner
val p = new HashPartitioner(12)
val data_rdd = parquetFileDF.rdd
val data_rdd802 = parquetFileDF802.rdd
val rddTaxiTrip = data_rdd.map(TaxiTrip.extract).repartition(12).cache()
val rddTaxiTrip2 = data_rdd802.map(TaxiTrip.extract).repartition(12).cache()
//val rddData = sc.parallelize(data_cached.take(10))
//rddData.collect()
//rddWeather.repartition(12).partitioner

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.HashPartitioner
p: org.apache.spark.HashPartitioner = org.apache.spark.HashPartitioner@c
data_rdd: org.apache.spark.rdd.RDD[org.apache.spark.sql.Row] = MapPartitionsRDD[9] at rdd at <console>:29
data_rdd802: org.apache.spark.rdd.RDD[org.apache.spark.sql.Row] = MapPartitionsRDD[15] at rdd at <console>:29
rddTaxiTrip: org.apache.spark.rdd.RDD[TaxiTrip] = MapPartitionsRDD[20] at repartition at <console>:30
rddTaxiTrip2: org.apache.spark.rdd.RDD[TaxiTrip] = MapPartitionsRDD[25] at repartition at <console>:30


### File di input
Per tutte le seguenti query verranno utilizzati due file, i quali verranno uniti successivamente attraverso una union: il primo contenente i dati per il mese di gennaio 2018, il secondo contente i dati di febbraio 2018. Alternativamente se si vuole prendere in considerazione più dati, è possibile cambiare il percorso specificato precedentemente, dal file parquet alla cartella contente tutti e 12 file (rappresentanti i mesi dell’anno). La grandezza di input è di circa 117 mb a file.

### Numero totale di record

In [8]:
rddTaxiTrip.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res16: Long = 8760687


### Numero totale di zone di partenza diverse

In [7]:
rddTaxiTrip.map(x => x.startloc).distinct().count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res11: Long = 7


### Query 1
Viaggi più lunghi in termine di km, in relazione con durata in minuti del viaggio

In [8]:
val query1 = rddTaxiTrip.map(x => (x.distance, (x.dropoff.getTime() - x.pickup.getTime())/(60*1000)))
val query1802 = rddTaxiTrip2.map(x => (x.distance, (x.dropoff.getTime() - x.pickup.getTime())/(60*1000)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

query1: org.apache.spark.rdd.RDD[(Double, Long)] = MapPartitionsRDD[30] at map at <console>:29
query1802: org.apache.spark.rdd.RDD[(Double, Long)] = MapPartitionsRDD[31] at map at <console>:29


In [9]:
val query1union = query1.union(query1802).cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

query1union: org.apache.spark.rdd.RDD[(Double, Long)] = UnionRDD[32] at union at <console>:30


In [10]:
query1union.sortByKey(false).take(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res12: Array[(Double, Long)] = Array((189483.84,1), (1061.2,9), (830.8,5), (484.91,674), (270.4,0), (267.7,308), (252.1,284), (251.64,1198), (213.44,195), (204.81,226))


### Query 2
Viaggi più costosi, in relazione con la somma tra la lunghezza e la durata in minuti del viaggio.
Si puo notare come i due valori siano direttamente proporzionali.
Aggiunto un filtro sulla zona di partenza

In [14]:
val query2 = rddTaxiTrip.filter(_.distance > 10).map(x => (x.fare, ((x.dropoff.getTime() - x.pickup.getTime())/(60*1000) + x.distance)))
val query2802 = rddTaxiTrip2.filter(_.distance > 10).map(x => (x.fare, ((x.dropoff.getTime() - x.pickup.getTime())/(60*1000) + x.distance)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

query2: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[50] at map at <console>:31
query2802: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[52] at map at <console>:31


In [15]:
val query2union = query2.union(query2802).cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

query2union: org.apache.spark.rdd.RDD[(Double, Double)] = UnionRDD[53] at union at <console>:32


In [16]:
query2union.sortByKey(false).take(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res20: Array[(Double, Double)] = Array((2409.0,1158.91), (1176.0,1449.6399999999999), (1020.5,430.81), (950.0,575.7), (935.5,371.9), (916.0,893.4), (700.01,55.3), (700.0,536.1), (690.0,268.5), (557.0,318.1))


### Query 3
Query per vedere i viaggi più lunghi in minuti in relazione ad una certa zona; groupbykey per vedere tutti i viaggi di una certa zona.
PartitionBy dopo il primo mapping per ottimizzare la compuitazione degli rdd, tenendo presente la regola di partizione descritta precedentemente.
Sort dopo il groupby per una computazione piu' veloce (meno lavoro fa computare essendo i record gia raggruppati per chiave)

Query per vedere la lunghezza media in minuti dei viaggi in base alla zona di partenza

In [11]:
val query3 = rddTaxiTrip.map(x => (x.startloc, (x.dropoff.getTime() - x.pickup.getTime())/(60*1000))).partitionBy(p)
val query3802 = rddTaxiTrip2.map(x => (x.startloc, (x.dropoff.getTime() - x.pickup.getTime())/(60*1000))).partitionBy(p)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

query3: org.apache.spark.rdd.RDD[(Long, Long)] = ShuffledRDD[37] at partitionBy at <console>:30
query3802: org.apache.spark.rdd.RDD[(Long, Long)] = ShuffledRDD[39] at partitionBy at <console>:30


In [12]:
val query3union = query3.union(query3802).cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

query3union: org.apache.spark.rdd.RDD[(Long, Long)] = PartitionerAwareUnionRDD[40] at union at <console>:30


In [13]:
query3union.groupByKey().map({case(k,v) => (v.sum/v.size,k)}).sortByKey(false).take(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res13: Array[(Long, Long)] = Array((45,2), (42,3), (42,4), (16,5), (14,1), (7,99), (1,6))


### Query 4
Map per group by location/time, reduceByKey per contare le occorrenze dei record, map scambiare chiave-valore per poter ordinare i valori per vedere le zone/periodi piu gettonati.

In [20]:
rddTaxiTrip.map(x => (x.startloc, 1)).reduceByKey((x,y) => x + y).map({case(k,v) => (v,k)}).sortByKey(false).collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res22: Array[(Int, Long)] = Array((8533295,1), (179273,2), (27735,5), (15129,3), (5080,4), (106,99), (69,6))


Le due seguenti query per visualizzare gli orari e giorni della settimana più utilizzati 

In [21]:
val query4 = rddTaxiTrip.map(x => (getDayTime(x.pickup.getTime()).get(Calendar.HOUR_OF_DAY), 1)).partitionBy(p)
val query4802 = rddTaxiTrip2.map(x => (getDayTime(x.pickup.getTime()).get(Calendar.HOUR_OF_DAY), 1)).partitionBy(p)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

query4: org.apache.spark.rdd.RDD[(Int, Int)] = ShuffledRDD[74] at partitionBy at <console>:33
query4802: org.apache.spark.rdd.RDD[(Int, Int)] = ShuffledRDD[76] at partitionBy at <console>:33


In [22]:
val query4union = query4.union(query4802).cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

query4union: org.apache.spark.rdd.RDD[(Int, Int)] = PartitionerAwareUnionRDD[77] at union at <console>:32


In [34]:
query4union.reduceByKey((x,y) => x + y).map({case(k,v) => (v,k)}).sortByKey(false).collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res30: Array[(Int, Int)] = Array((1134621,18), (1071054,19), (1016000,17), (967043,15), (950874,20), (948483,14), (944772,21), (903898,16), (895293,13), (884236,22), (883709,12), (824693,9), (822848,8), (819874,11), (796772,10), (694060,23), (654813,7), (512199,0), (384824,6), (362372,1), (269327,2), (194167,3), (166529,5), (151045,4))


In [55]:
query4union.reduceByKey((x,y) => x + y).map({case(k,v) => (v,k)}).sortByKey(false).collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res46: Array[(Int, Int)] = Array((1134621,18), (1071054,19), (1016000,17), (967043,15), (950874,20), (948483,14), (944772,21), (903898,16), (895293,13), (884236,22), (883709,12), (824693,9), (822848,8), (819874,11), (796772,10), (694060,23), (654813,7), (512199,0), (384824,6), (362372,1), (269327,2), (194167,3), (166529,5), (151045,4))


Salvataggio in file csv per successiva visualizzazione dati con Tableau 

In [ ]:
val tableau = rddTaxiTrip.map(x => (getDayTime(x.pickup.getTime()).get(Calendar.DAY_OF_WEEK), 1)).
                    reduceByKey((x,y) => x + y).map({case(k,v) => (v,k)}).sortByKey(false)

In [ ]:
//import org.apache.spark.sql.SaveMode
//tableau.coalesce(1).toDF().write.format("csv").mode(SaveMode.Overwrite).save("s3a://"+bucketname+"/datasets/project/output/mostTripsByHour")

### Query 5
Distanza media percorsa dai taxi in un singolo viaggio, in relazione alla zona.
Aggregazione eseguita prima dell'union effettivo per ridurre i tempi di esecuzione.

In [ ]:
val query5 = rddTaxiTrip.filter(_.distance < 100).map(x => (x.startloc, x.distance)).
            aggregateByKey((0.0,0.0))((a,v)=>(a._1+v,a._2+1), (a1,a2)=>(a1._1+a2._1,a1._2+a2._2)).partitionBy(p)
val query5802 = rddTaxiTrip2.filter(_.distance < 100).map(x => (x.startloc, x.distance)).
            aggregateByKey((0.0,0.0))((a,v)=>(a._1+v,a._2+1), (a1,a2)=>(a1._1+a2._1,a1._2+a2._2)).partitionBy(p)

In [26]:
val query5union = query5.union(query5802).cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

query5union: org.apache.spark.rdd.RDD[(Long, (Double, Double))] = PartitionerAwareUnionRDD[95] at union at <console>:32


In [27]:
query5union.map({case(k,v)=>(k,v._1/v._2)}).map({case(k,v) => (v,k)}).sortByKey(false).collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res25: Array[(Double, Long)] = Array((18.272109870868892,4), (17.795655769988183,4), (17.03518382578527,2), (16.993668001025206,2), (16.66991076150185,3), (16.37588990575058,3), (5.870928177194196,5), (5.345708166422089,5), (2.4477249319437977,1), (2.4389944840742377,1), (1.0672463768115945,6), (0.9196825396825397,6), (0.05377358490566038,99), (0.027522935779816515,99))


### Query 6
Numero di viaggi in base alla fascia oraria, per vedere le più trafficate per passeggeri.

In [28]:
val query6 = rddTaxiTrip.map(x => (getDayTime(x.pickup.getTime()).get(Calendar.HOUR_OF_DAY), x.passenger)).partitionBy(p)
val query6802 = rddTaxiTrip2.map(x => (getDayTime(x.pickup.getTime()).get(Calendar.HOUR_OF_DAY), x.passenger)).partitionBy(p)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

query6: org.apache.spark.rdd.RDD[(Int, Long)] = ShuffledRDD[102] at partitionBy at <console>:33
query6802: org.apache.spark.rdd.RDD[(Int, Long)] = ShuffledRDD[104] at partitionBy at <console>:33


In [29]:
val query6union = query6.union(query6802).cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

query6union: org.apache.spark.rdd.RDD[(Int, Long)] = PartitionerAwareUnionRDD[105] at union at <console>:32


In [30]:
query6union.groupByKey().map({case(k,v) => ((math floor (v.sum.toDouble/v.size) * 100) / 100,k)}).sortByKey(false).collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res26: Array[(Double, Int)] = Array((1.65,1), (1.65,2), (1.65,3), (1.64,0), (1.64,4), (1.64,22), (1.64,23), (1.63,21), (1.61,19), (1.61,20), (1.6,15), (1.6,17), (1.6,18), (1.59,12), (1.59,13), (1.59,14), (1.59,16), (1.59,11), (1.57,10), (1.55,8), (1.55,9), (1.54,5), (1.54,7), (1.51,6))


Infine i file parquet possono anche essere utlizzati per creare delle "view" temporanee e essere usati per query SQL

In [56]:
parquetFileDF.createOrReplaceTempView("parquetFile")
val parkDF = spark.sql("SELECT * FROM parquetFile")
parkDF

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

parkDF: org.apache.spark.sql.DataFrame = [VendorID: bigint, tpep_pickup_datetime: timestamp ... 17 more fields]
res48: org.apache.spark.sql.DataFrame = [VendorID: bigint, tpep_pickup_datetime: timestamp ... 17 more fields]
